In [1]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt


In [2]:
n_qubits = 3

qubits = cirq.GridQubit.rect(1, n_qubits)
g=1
J=1.21
observable = [-float(0.5*g)*cirq.Z.on(q) for q in qubits] 
for q in range(len(qubits)):
    observable.append(-float(0.5*J)*cirq.X.on(qubits[q])*cirq.X.on(qubits[(q+1)%len(qubits)]))
circuit = []

parametrized_unitary = [cirq.rz, cirq.rx, cirq.rz]

indexed_cnots = {}
cnots_index = {}
count = 0
for control in range(n_qubits):
    for target in range(n_qubits):
        if control != target:
            indexed_cnots[str(count)] = [control, target]
            cnots_index[str([control,target])] = count
            count += 1
number_of_cnots = len(indexed_cnots)

def append_to_circuit(ind, circuit, params, new_index=False):
    if ind < number_of_cnots:
        control, target = indexed_cnots[str(ind)]
        circuit.append(cirq.CNOT.on(qubits[control], qubits[target]))
        return circuit, params
    else:
        qubit = qubits[(ind-number_of_cnots)%n_qubits]
        for par, gate in zip(range(3),parametrized_unitary):
            new_param = "th_"+str(len(params))
            params.append(new_param)
            circuit.append(gate(sympy.Symbol(new_param)).on(qubit))
        return circuit, params
    
def give_circuit(lista):
    circuit, symbols = [], []
    for k in lista:
        circuit, symbols = append_to_circuit(k,circuit,symbols)
    circuit = cirq.Circuit(circuit)
    return circuit, symbols

In [ ]:
for k in tqdm(range(30)):
    circuit,symbols = give_circuit([number_of_cnots,number_of_cnots+1,number_of_cnots+2])

    circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
    output = tfq.layers.Expectation(backend=cirq.DensityMatrixSimulator(noise=cirq.depolarize(0.00001)))(
    circuit_input,
    symbol_names=symbols,
    operators=tfq.convert_to_tensor([observable]),
    initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!

    model = tf.keras.Model(inputs=circuit_input, outputs=output)
    adam = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=adam, loss='mse')

    tfqcircuit = tfq.convert_to_tensor([circuit])
    qoutput = tf.ones((1, 1))*-2.*n_qubits
    
    model.set_weights([tf.random.uniform((len(symbols),))])
    model.fit(x=tfqcircuit, y=qoutput,validation_steps=None,
              steps_per_epoch=1, batch_size=1, epochs=2, verbose=1, 
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])

  0%|          | 0/30 [00:00<?, ?it/s]

Train on 1 samples
Epoch 1/2
1/1 [==============================] - 2s 2s/sample - loss: 33.4428
Epoch 2/2
1/1 [==============================] - 1s 506ms/sample - loss: 33.4291


  3%|▎         | 1/30 [00:03<01:51,  3.84s/it]

Train on 1 samples
Epoch 1/2
1/1 [==============================] - 1s 1s/sample - loss: 33.3266
Epoch 2/2
1/1 [==============================] - 1s 803ms/sample - loss: 33.3152


  7%|▋         | 2/30 [00:05<01:32,  3.30s/it]

...

 20%|██        | 6/30 [00:13<00:53,  2.24s/it]

Train on 1 samples
Epoch 1/2
1/1 [==============================] - 1s 1s/sample - loss: 33.3245
Epoch 2/2
1/1 [==============================] - 1s 608ms/sample - loss: 33.3111


 23%|██▎       | 7/30 [00:15<00:50,  2.18s/it]

Train on 1 samples
Epoch 1/2


Process ForkPoolWorker-123:
Process ForkPoolWorker-122:
Process ForkPoolWorker-124:
Traceback (most recent call last):
Process ForkPoolWorker-121:
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cooper-cooper/.local/lib/python3.6/site-packages/multiprocess/queues.py", li

In [ ]:
np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))

In [ ]:
circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
output = tfq.layers.Expectation()(
circuit_input,
symbol_names=symbols,
operators=tfq.convert_to_tensor([observable]),
initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!

model = tf.keras.Model(inputs=circuit_input, outputs=output)
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss='mse')
tfqcircuit = tfq.convert_to_tensor([circuit])
qoutput = tf.ones((1, 1))*-2.*n_qubits
model.fit(x=tfqcircuit, y=qoutput, batch_size=1, epochs=250, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])

In [ ]:
np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))